# Setup

In [1]:
# Shared "communication" directory on Drive.
shared_directory = '/content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/'

# trained_model = '/content/drive/My Drive/PredatorEye/saved_models/20211220_1404'
# trained_model = '/content/drive/My Drive/PredatorEye/saved_models/20211222_1044'
# use fresh copy on Jan 1: 20220101_0954
trained_model = '/content/drive/My Drive/PredatorEye/saved_models/20220101_0954'
model = []

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
other_suffix = ".jpeg"

fcd_image_size = 1024
fcd_disk_size = 201

import time
from os import listdir
from os import remove
from os.path import join
from os.path import split
from os.path import isfile
from tensorflow import keras

from PIL import Image
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')



TensorFlow version: 2.7.0


# After c++ tests

In [2]:

def run_test():
    print('Start run in', shared_directory )
    list_unexpected_files(shared_directory)
    step = 0;
    while True:
        performStep(step, shared_directory)
        step += 1

def performStep(step, directory):
    waitForReply(step, shared_directory)
    print('Write file', step)
    writeTestFile(step, shared_directory)
    deleteMyFile(step - 1, shared_directory)

# Mock version of writing file for given step.
def writeTestFile(step, directory):
    pathname = makeMyPathname(step, directory)
    file = open(pathname,"w")
    #
    # file.write(str(step))
    # file.write(str(step) + " " + str(step))

    image = Image.open(makeOtherPathname(step, directory))
    # new_size = (fcd_image_size, fcd_image_size)
    # pixels = asarray(image.resize(new_size, PIL.Image.LANCZOS),
    #                     dtype=np.float32)        
    pixel_tensor = np.asarray(image, dtype=np.float32)
    print('pixel_tensor.shape', pixel_tensor.shape)
    predict = model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    x = predict[0]
    y = predict[1]
    # print(i, ": (", x, ",", y, ")")
    #
    # image_size = 128 # temp, derive from data
    # file.write(str(x * image_size) + " " + str(y * image_size))
    #
    response_string = str(x) + " " + str(y)
    # print('response_string', response_string)
    # print('x =', x, ', y =', y, ', response_string =', response_string)
    print('response_string' + "'" + response_string + "'")
    file.write(response_string)
    file.close()
    #
    print("wrote test file  ", makeMyPathname(step, directory))

# Delete the given file, usually after having written the next one.
def deleteMyFile(step, directory):
    path = makeMyPathname(step, directory)
    if isfile(path):
        remove(path)

# Form pathname for file of given step number from the "other" agent.
def makeOtherPathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for file of given step number from "this" agent.
def makeMyPathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

# Wait until other agent's file for given step appears.
def waitForReply(step, directory):
    print('start waiting for', makeOtherPathname(step, directory))
    start_time = time.time()
    while not isFilePresent(makeOtherPathname(step, directory)):
        time.sleep(2)  # wait 2 sec
    print('done waiting for ', makeOtherPathname(step, directory))
    print('Elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists() but for unknown reasons, that does not
# seem to work for newly created files on G Drive.
#
# TODO Why? This version works on G Drive, but it seems simply
#      calling fs::exists() should be enough.
#
def isFilePresent(file):
    result = False
    (directory, filename) = split(file)
    for i in listdir(directory):
        if i == filename:
            result = True
    return result

# isFilePresent(comm_directory + 'foo.txt')
# isFilePresent(comm_directory + 'bar.txt')
# waitForReply(1, comm_directory)

# Actually I guess the counterparty may have already written its first...
def list_unexpected_files(directory):
    directory_contents = listdir(directory)
    if directory_contents:
        print('Unexpected files:', directory_contents)

# Read pre-trained model

In [3]:
# read model

print('Reading pre-trained model from:', trained_model)
# model = keras.models.load_model(trained_model)
# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373

# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }

# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

# 20220101 can I now compile the reloaded model?
# model = keras.models.load_model(trained_model,
#                                 custom_objects=dependencies,
#                                 compile=False)
model = keras.models.load_model(trained_model, custom_objects=dependencies)

Reading pre-trained mode from: /content/drive/My Drive/PredatorEye/saved_models/20220101_0954


# Run test

In [4]:
run_test()

Start run in /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/
Unexpected files: ['camo_0.jpeg']
start waiting for /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_0.jpeg
done waiting for  /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_0.jpeg
Elapsed time: 0 seconds.
Write file 0
pixel_tensor.shape (128, 128, 3)
x = 0.5337393 , y = 0.59821725 , response_string = 0.5337393 0.59821725
wrote test file   /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/find_0.txt
start waiting for /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_1.jpeg
done waiting for  /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_1.jpeg
Elapsed time: 36 seconds.
Write file 1
pixel_tensor.shape (128, 128, 3)
x = 2.2064006 , y = 2.2662778 , response_string = 2.2064006 2.2662778
wrote test file   /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/find_1.txt
start waiting for /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd

KeyboardInterrupt: ignored